**Group Project Report: Pulsar Star Classification**
===================

# Introduction

Pulsars or Pulse Stars are highly magnetized neutron stars that radiate radio-wave emissions of two opposing narrow beams of light that are spinning, which we perceive on earth as blinking (Cofield, 2016). Each pulsar star emits a different emission pattern that varies with each of its rotations, therefore whenever there is a signal detection, or “candidate”, it is averaged over several rotations of the pulse star (Lyon, 2017). Each of these candidates are potential descriptions of a real pulse star. Radio frequency interference and noise causes is responsible for most detections, the true pulse star signals are challenging to extract (Lyon, 2017).

Our data set includes pulsar candidate data. Using this data, we will attempt to predict which of these candidates are legitimate pulsar stars and which are not. Our question is thus: Can we differentiate legitimate pulsar stars from interference and noise using a classification model?

Our Data set can be found here: https://archive.ics.uci.edu/ml/datasets/HTRU2

This data set includes 9 variables (9 columns), and approximately 18000 rows. 

| Variables |  Definitions |
|----------------|------------|
| <p align="left">Mean IP| <p align="left">Mean of the integrated profile 
|
| <p align="left">Standard deviation IP| <p align="left">Standard deviation of the integrated profile|
 | <p align="left">Excess kurtosis IP| <p align="left">Excess kurtosis of the integrated profile|
| <p align="left">Skewness IP| <p align="left">Skewness of the integrated profile|
 | <p align="left">Mean DM-SNR curve| <p align="left">Mean of the Dispersion Measure - Signal to noise ratio curve|
| <p align="left">Standard deviation DM-SNR curve| <p align="left">Standard deviation of the Dispersion Measure - Signal to noise ratio curve|
| <p align="left">Excess kurtosis DM-SNR curve| <p align="left">Excess kurtosis of the Dispersion Measure - Signal to noise ratio curve|
| <p align="left">Skewness DM-SNR curve|  <p align="left">Skewness of the of the Dispersion Measure - Signal to noise ratio curve|
| <p align="left">Class | <p align="left">Whether or not an observation is a pulsar star or not. If Class = 0, the observation is not a pulsar star, while if Class = 1, the observation is a pulsar star.|

**The integrated profile variables refer to a method used to describe the radiation pulsar stars emit.**

Mean of the integrated profile.
Standard deviation of the integrated profile.
Excess kurtosis of the integrated profile.
Skewness of the integrated profile.
Mean of the DM-SNR curve.
Standard deviation of the DM-SNR curve.
Excess kurtosis of the DM-SNR curve.
Skewness of the DM-SNR curve.
Class

The integrated profile variables refer to a method used to describe the radiation pulsar stars emit.
    Individual pulses vary in shape, and their radiation is detected as individual photons which arrive infrequently (Lyne & Graham Smith, 2012). The timing of the pulse’s arrival also varies across different radio frequencies. The delay between frequencies is known as dispersion (Krishnan, 2020). 
    
DM-SNR, or Dispersion Measure - Signal to noise ratio curves are created to adjust for the effect of these frequency delays (Krishnan, 2020). The variables 1-8 in this dataset thus describe the mean, skewness, standard deviation, and excess kurtosis of the integrated profile and DM-SNR curves. The Class variable refers to whether or not an observation is a pulsar star or not. 0 = the observation is not a pulsar star,  1 = the observation is a pulsar star.

    

# Methods and Results

### Preliminary Exploratory Data Analysis

In [ ]:
## install packages
#install.packages("tidyverse")
#install.packages("repr")
#install.packages("tidymodels")
#install.packages("cowplot")

In [ ]:
## RUN THIS CELL BEFORE CONTINUING 
#Note: if the packages not installed, uncomment the previous cell and run it
## load packages
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)

In [ ]:
## LOADING DATA FROM THE WEB##

#link to the zip file we want to download
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip"

#Create temporary file to store zip file and download zip file
temp <- tempfile()
download.file(url, temp)

#read dataset (HTRU_2.csv) from zip file (temp) + give each column appropriate name
pulsar_data <- read_csv(unz(temp, "HTRU_2.csv"), col_names = c("Mean IP", 
                                                         "Standard deviation IP", 
                                                         "Excess kurtosis IP", 
                                                         "Skewness IP",
                                                         "Mean DM-SNR curve",
                                                         "Standard deviation DM-SNR curve",
                                                         "Excess kurtosis DM-SNR curve",
                                                         "Skewness DM-SNR curve",
                                                         "Class"))
#Note: IP = Integrated Profile

#delete temporary file (because no longer needed)
unlink(temp)

#display the first 6 rows of our dataset
head(pulsar_data)

In [ ]:
## CLEANING & WRANGLING DATA INTO A TIDY FORMAT ##

# make the class column (that determines whether an observation is a pulsar or not) as a factor
pulsar_data <- pulsar_data |> 
        mutate(Class = as_factor(Class))

#replace spaces in column names by dots
colnames(pulsar_data) = make.names(colnames(pulsar_data))

#display the first 6 rows of our now tidy dataset
head(pulsar_data)

In [ ]:
## EXPLORATORY DATA ANALYSIS ##

#DO NOT CHANGE THIS (this seed value will makes our results reproductible)
set.seed(1)

# splitting data into training and testing data
pulsar_split <- initial_split(pulsar_data, prop = .75, strata = Class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split)

## Summarize training data: 

# 1. dimensions of training data
pulsar_train_dimensions <- pulsar_train |>
        dim()
pulsar_train_dimensions

In [ ]:
# 2. number and proportion of observations in each class type
pulsar_train_summary <- pulsar_train |> 
  group_by(Class) |> 
  summarize(n = n(),
            proportion = n()/13423) #ref. to dimentions above (nb of rows corresponds to nb of observations)

pulsar_train_summary

In [ ]:
# 3. means of the predictor variables
pulsar_predictor_means <- pulsar_train |> 
  select(-Class) |> 
  summarize(across(everything(), mean))

pulsar_predictor_means

In [ ]:
# 4. number of rows with missing data

pulsar_missing_data <- pulsar_train |> 
    filter_at(pulsar_train |> colnames() |> all_of() |> vars(), any_vars(is.na(.)))

number_of_rows_missing <- nrow(pulsar_missing_data)
number_of_rows_missing  # there are no missing data in this dataset

In [ ]:
## EXPLORATORY DATA ANALYSIS: VISUALIZATIONS ##

# these boxplots show the distribution of observations for each variable and how that relates to the Class of the observation
pulsar_boxplot_IPmean <- pulsar_train |> 
      ggplot(aes(x = Mean.IP, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Mean Integrated Profile" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_DMSNRmean <- pulsar_train |> 
      ggplot(aes(x = Mean.DM.SNR.curve, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Mean DM-SNR Curve" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_SDIP <- pulsar_train |> 
      ggplot(aes(x = Standard.deviation.IP, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Standard deviation Integrated Profile" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_SD_DMSNR <- pulsar_train |> 
      ggplot(aes(x = Standard.deviation.DM.SNR.curve, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Standard deviation DM-SNR Curve" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_exkur_IP <- pulsar_train |> 
      ggplot(aes(x =Excess.kurtosis.IP , y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Excess kurtosis Integrated Profile" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_exkur_DMSNR <- pulsar_train |> 
      ggplot(aes(x = Excess.kurtosis.DM.SNR.curve , y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Excess kurtosis DM-SNR Curve" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_skew_IP <- pulsar_train |> 
      ggplot(aes(x = Skewness.IP, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Skewness Integrated Profile" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

pulsar_boxplot_skew_DMSNR <- pulsar_train |> 
      ggplot(aes(x = Skewness.DM.SNR.curve, y = Class, fill = Class))+
      geom_boxplot()+
      labs( x = "Skewness DM-SNR Curve" ,
            y =  "Class") + 
      coord_flip()+ 
      scale_fill_discrete(labels = c( "Not a Pulsar Star", "Pulsar Star"))+
      theme(text = element_text(size = 20))

options(repr.plot.width=20, repr.plot.height=20)

plot_grid(pulsar_boxplot_IPmean, pulsar_boxplot_DMSNRmean, pulsar_boxplot_SDIP, pulsar_boxplot_SD_DMSNR,pulsar_boxplot_exkur_IP, pulsar_boxplot_exkur_DMSNR, pulsar_boxplot_skew_IP, pulsar_boxplot_skew_DMSNR )


These 8 plots compare the disributions of observations between means, standard deviation, excess kurtosis and skewness of the DM-SNR profiles and Integrated Profiles across the 2 Classes. For all 8 plots, the potential predictive variable is correlated with Class, although this relationship is weaker for the skewness of the DM-SNR and Integrated Profile variables. However, the difference in medians is most significant for the Mean Integrated Profile, Standard deviation DM-SNR Curve, Excess kurtosis Integrated Profile variables and thus these will be used as our predictive variables. Many of the variables have a large number of outliers, and smaller differences between median values, and were thus not used as predictive variables.

### K Nearest Neighbors Classification

 In order to perform our analysis, we used the mean integrated profile, excess kurtosis of the integrated profile, and standard deviation DM-SNR Curve variables as predictive variables. As discussed in the exploratory data analysis, **these 6 variables show a clear relationship between their value and the Class of the observation**, while the other variables show a weaker relationship. 
 
 In order to perform our analysis, we **split the data into training and testing data** as shown in the exploratory analysis section. This step was important to ensure our model did not "see" the testing data during training, as this would result in an erroneously high accuracy value. We then used **K- Nearest Neighbors Classification** to create a model which classifies our testing data as a pulsar star or not.  We created a model using our predictive variables and used a **5-fold cross-validation technique** to determine the K value that gives us the greatest predictive accuracy. Finally, we **re-trained our model** with the K-value obtained from the cross-validation process and **evaluated our model** using the testing data.  
 
 We used K-NN Classification because our question revolved around whether or not a given observation was a pulsar star or not. Because our question involves determining the category of an observation, rather than predicting a numerical value, classification was a more appropriate technique than regression. 



In [ ]:
# create KNN model
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
        set_engine("kknn") |>
        set_mode("classification")

# create a recipe
recipe <- recipe(Class ~ Mean.IP+Standard.deviation.DM.SNR.curve+Excess.kurtosis.IP , data = pulsar_train) |>
        step_scale(all_predictors()) |>
        step_center(all_predictors())


In [ ]:
# 5 fold cross-validation
vfold_train <- vfold_cv(pulsar_train, v = 5, strata = Class)

In [ ]:
# fit and tune the model to test different values of k
pulsar_fit <- workflow() |> 
    add_recipe(recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = vfold_train, grid = 10) |>
    collect_metrics()

#pull the k value with the highest accuracy for use in the final model
kmin <- pulsar_fit |>
        filter(.metric == "accuracy") |>
        filter(mean == max(mean)) |>
        select(neighbors) |>
        tail(1)|>
        pull()
kmin

k = 14 had the highest accuracy and thus was used in the final model training.

In [ ]:
## re-train model with k-value obtained from cross-validation

# create KNN model
knn_spec2 <-nearest_neighbor(weight_func = "rectangular", neighbors = kmin) |>
        set_engine("kknn") |>
        set_mode("classification")

#fit the model 
pulsar_fit2 <- workflow() |> 
    add_recipe(recipe) |>
    add_model(knn_spec2) |>
    fit(data = pulsar_train)


In [ ]:
## evaluate model using testing data
pulsar_predictions <- predict(pulsar_fit2 , pulsar_test) |>
    bind_cols(pulsar_test) |>
    select(.pred_class, Class)

head(pulsar_predictions)

# pull the accuracy metrics 
pulsar_metrics <- pulsar_predictions |>
  metrics(truth = Class, estimate = .pred_class) |>
    filter(.metric == "accuracy") |>
    select(.estimate)

pulsar_metrics

# create a confusion matrix
pulsar_conf_mat <-pulsar_predictions |>
    conf_mat(truth = Class , estimate = .pred_class)

pulsar_conf_mat

Our model has an accuracy value of 98.01%, which is very good. The confusion matrix summarizes the prediction results, indicating how many correct and incorrect classifications the model made for each class. The model correctly classified 4034 non-pulsar observations, and incorrectly classified 64 non-pulsar observations as pulsar stars. The model correctly classified 352 pulsar stars, and incorrectly classified 25 pulsar stars as non-pulsar stars.

In [ ]:
## Visualization

# For each variable, create 3 vectors that have 60 values, from the minimum to the maximum value of that variable
mean.ip_grid <- seq(min(pulsar_train$Mean.IP), 
                    max(pulsar_train$Mean.IP),
                    length.out = 60)

sd.dmsnr_grid <- seq(min(pulsar_train$Standard.deviation.DM.SNR.curve), 
                     max(pulsar_train$Standard.deviation.DM.SNR.curve),
                     length.out = 60)

ek.ip_grid <- seq(min(pulsar_train$Excess.kurtosis.IP), 
                  max(pulsar_train$Excess.kurtosis.IP),
                  length.out = 60)

# Create a database of all the possible combinations of the three vectors; this represents a 3D grid of size 60x60x60 (= 180,000 observations)
grid <- as_tibble(expand.grid(Mean.IP = mean.ip_grid,
                              Standard.deviation.DM.SNR.curve = sd.dmsnr_grid,
                              Excess.kurtosis.IP = ek.ip_grid))

In [ ]:
# predict each observation of the grid; will help us visualize the model's behavior
predict_grid <- pulsar_fit2 |>
        predict(grid) |>
        bind_cols(grid) |>
        rename(Class = .pred_class)

In [ ]:
# Create a function called "get_mode" that produces the statistical mode of a variable (here, it will be used for Class) 
get_mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

# Data tidying for each pair of variable: abstraction of the unwanted variable to keep the two variables of interest. Done replacing the Class by its mode.

    # Mean IP & Standard Deviation DM DNR curve
m_vs_sd <- predict_grid |>
        group_by(Mean.IP,Standard.deviation.DM.SNR.curve) |>
        summarize(Class_mode = get_mode(Class),
                  Mean.IP = unique(Mean.IP),
                  Standard.deviation.DM.SNR.curve = unique(Standard.deviation.DM.SNR.curve))

    # Excess kurtosis IP & Standard Deviation DM DNR curve
ek_vs_sd <- predict_grid |>
        group_by(Excess.kurtosis.IP,Standard.deviation.DM.SNR.curve) |>
        summarize(Class_mode = get_mode(Class),
                  Excess.kurtosis.IP = unique(Excess.kurtosis.IP),
                  Standard.deviation.DM.SNR.curve = unique(Standard.deviation.DM.SNR.curve))

    # Mean IP & Excess kurtosis IP
m_vs_ek <- predict_grid |>
        group_by(Mean.IP,Excess.kurtosis.IP) |>
        summarize(Class_mode = get_mode(Class),
                  Mean.IP = unique(Mean.IP),
                  Excess.kurtosis.IP = unique(Excess.kurtosis.IP))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

# Creation of plots for each variable pair by overlaying the data over the predicted grid points; This allows us to see the model's behavior for all values.

    # Mean IP & Standard Deviation DM DNR curve
m_vs_sd_plot <- ggplot() +
        geom_point(data = m_vs_sd,
                  mapping = aes(x = Mean.IP,
                                y = Standard.deviation.DM.SNR.curve,
                                color = Class_mode),
                                alpha = .01,
                                size = 25) +
        geom_point(data = pulsar_data,
                   mapping = aes(x = Mean.IP,
                                 y = Standard.deviation.DM.SNR.curve,
                                 color = Class),
                                 alpha = .5) +
        labs(color = "Pulsar?", x = "Mean IP", y = "Standard Deviation DM-SNR curve", title = "Model viz for the variables \nMean IP and Standard Deviation DM-SNR curve") +
        scale_color_manual(labels = c("Not Pulsar", "Pulsar"), 
                     values = c("#005AB5", "#D41400")) +       #Colorblind friendly colors (source: https://davidmathlogic.com/colorblind/#%23994F00-%23006CD1)
        theme(text = element_text(size = 22))

    # Excess kurtosis IP & Standard Deviation DM DNR curve
ek_vs_sd_plot <- ggplot() +
        geom_point(data = ek_vs_sd,
                  mapping = aes(x = Excess.kurtosis.IP,
                                y = Standard.deviation.DM.SNR.curve,
                                color = Class_mode),
                                alpha = .01,
                                size = 25) +
        geom_point(data = pulsar_data,
                   mapping = aes(x = Excess.kurtosis.IP,
                                 y = Standard.deviation.DM.SNR.curve,
                                 color = Class),
                                 alpha = .5) +
        labs(color = "Pulsar?", x = "Excess Kurtosis IP", y = "Standard Deviation DM-SNR curve", title = "Model viz for the variables \nExcess Kurtosis IP and Standard Deviation DM-SNR curve") +
        scale_color_manual(labels = c("Not Pulsar", "Pulsar"), 
                     values = c("#005AB5", "#D41400")) +       #Colorblind friendly colors (source: https://davidmathlogic.com/colorblind/#%23994F00-%23006CD1)
        theme(text = element_text(size = 22))

    #Mean IP & Excess kurtosis IP
m_vs_ek_plot <- ggplot() +
        geom_point(data = m_vs_ek,
                  mapping = aes(x = Mean.IP,
                                y = Excess.kurtosis.IP,
                                color = Class_mode),
                                alpha = .01,
                                size = 25) +
        geom_point(data = pulsar_data,
                   mapping = aes(x = Mean.IP,
                                 y = Excess.kurtosis.IP,
                                 color = Class),
                                 alpha = .5) +
        labs(color = "Pulsar?", x = "Mean IP", y = "Excess Kurtosis IP", title = "Model viz for the variables \nMean IP and Excess Kurtosis IP") +
        scale_color_manual(labels = c("Not Pulsar", "Pulsar"), 
                     values = c("#005AB5", "#D41400")) +       #Colorblind friendly colors (source: https://davidmathlogic.com/colorblind/#%23994F00-%23006CD1)
        theme(text = element_text(size = 22))

options(repr.plot.width = 20, repr.plot.height = 15)
plot_grid(m_vs_sd_plot, ek_vs_sd_plot, m_vs_ek_plot, ncols = 3)

# Discussion

Our analysis determined that we could predict whether an observation was a pulsar star or not with 98.01% accuracy, which is very high. This suggests our model is very good, and that the predictive variables we used are strong predictive variables. We expected that K-NN Classification would be a useful method for determining if an observation is a pulsar star or not, in part because it requires few assumptions about what the data looks like. However, we did not expect our accuracy to be so high. This is potentially due to the large size of the dataset used; there were 13,423 observations in our training set. 

Our findings could be impactful because if a model is sufficiently accurate at predicting the class of an observation, then a model such as ours could be used in place of all-sky surveys, a method for using a telescope to find pulsar stars (Cofield, 2016). This would significantly reduce the effort required to identify pulsar stars.

We expect that future potential questions might include the following:
- Do more optimal configurations of predictive variables than we used in our analysis exist, and if so, what are they?
- What are alternative ways we can predict the class of observations from potential pulsar star data, and how might those alternative methods change the accuracy of the predictions? 
- Are there other predictive variables we could obtain which would lead to better predictions of class? 
- Could we further classify non-pulsar stars into certain groups using this data?
- How do pulsar stars differ from other neutron stars in terms of their emissions patterns?
- Can we predict the size or age of a pulsar star using the predictive variables in our dataset?



**References**

Cofield, C. (2016, April 22). What are pulsars? Space.com. Retrieved October 27, 2022, from https://www.space.com/32661-pulsars.html 

Lyne, A., & Graham-Smith, F. (2012). 15 Integrated pulse profiles. In Pulsar Astronomy (Cambridge Astrophysics, pp. 207-229). Cambridge: Cambridge University Press. doi:10.1017/CBO9780511844584.016

Lyon, R.J. HTRU2. DOI: 10.6084/m9.figshare.3080389.v1.

Lyon, R.J, Stappers, B.W., Cooper, S., Brooke, J.M, Knowles, J.D. (2016). Fifty Years of Pulsar Candidate Selection: From simple filters to a new principled real-time classification approach, *Monthly Notices of the Royal Astronomical Society 459* (1), 1104-1123. DOI: 10.1093/mnras/stw656

Krishnan, Sowmya. (2020, September 27). Pulsar candidate classification. Medium. https://medium.com/swlh/pulsar-candidate-classification-a1805a369e17